<a href="https://colab.research.google.com/github/thomas0913/BlockChain_BasicCase/blob/main/BlockChain_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#定義區塊鏈格式與架構
###之中包含了區塊鏈必要的定義元素

*   交易格式

In [24]:
#交易定義
class Transaction:
  def __init__(self,sender,receiver,amounts,fee,message): 
    self.sender = sender    #發送者，同時check帳戶餘額是否足夠
    self.receiver = receiver  #接收者，通常直接收款
    self.amounts = amounts   #金額數
    self.fee = fee       #手續費
    self.message = message   #註記，generally for receiver

*   區塊格式



In [25]:
#區塊定義
class Block:
  def __init__(self,previous_hash,difficulty,miner,miner_rewards):
    self.previous_hash = previous_hash  #前一區塊之雜湊值，為了加密
    self.hash = ''            #當前區塊雜湊值，目前區塊計算後之雜湊值
    self.difficulty = difficulty     #當前難度
    self.nonce = 0            #能解開上個區塊鎖的鑰匙
    self.timestamp = int(time.time())   #區塊產生時之時間戳，調整挖礦難度時會用到
    self.transactions = []        #交易紀錄(for all)
    self.miner = miner          #挖掘礦工(誰挖的)
    self.miner_rewards = miner_rewards  #礦工獎勵，區塊產出時分給礦工的獎勵


*   區塊鏈架構





In [26]:
#區塊鏈定義
class BlockChain:
  def __init__(self):
    self.adjust_difficulty_blocks = 10 #難度調節區塊數，每多少區塊調節一次
    self.difficulty = 1        #當前難度
    self.block_time = 30        #出塊時間，理想上多久能夠產出一個區塊
    self.mining_rewards = 10      #挖礦獎勵，獎勵挖礦者的金額多寡
    self.block_limitation = 32     #區塊容量，每一區塊能夠容納的交易上限
    self.chain = []           #區塊鏈，目前鏈中儲存的所有區塊
    self.pending_transactions = []   #等待中的交易(因區塊鏈能吞吐的交易量有限)

#產生創世塊&挖掘新區塊
###1.   產生雜湊值(HASH)
###2.   產生創世塊
###3.   放置交易明細至新區塊中
###4.   挖掘新區塊



* 產生雜湊值(HASH)

In [27]:
import hashlib

class BlockChain:
  #負責把交易明細轉換成字串
  def transaction_to_string(self, transaction):
    transaction_dict = {
        'sender': str(transaction.sender),
        'receiver': str(transaction.receiver),
        'amounts': transaction.amounts,
        'fee': transaction.fee,
        'message': transaction.message
    }
    return str(transaction_dict)

  #負責把區塊記錄內的所有交易明細轉換成一個字串
  def get_transactions_string(self, block):
    transaction_str = ''
    for transaction in block.transactions:
      transaction_str += self.transaction_to_string(transaction)
    return transaction_str

  #負責依據這四筆資料產生相對應的雜湊值
  def get_hash(self, block, nonce):
    s = hashlib.sha1()
    s.update(
      (
         block.previous_hash           #前一區塊之雜湊值
         + str(block.timestamp)         #區塊產生當下的時間戳
         + self.get_transactions_string(block)  #區塊內所有之交易明細
         + str(nonce)              #挖掘中的nonce值
      ).encode("utf-8")
    )
    h = s.hexdigest()
    return h

* 產生創世塊

In [28]:
class BlockChain:
  #開始部屬區塊鏈所產生之第一個區塊，無任何交易紀錄且為無任何資料的空區塊
  def create_genesis_block(self):
    print("Create genesis block...")

    #定義創世塊 = Block(前一區塊雜湊值,預設難度,礦工姓名,預設挖礦獎勵)
    new_block = Block('Hello World!', self.difficulty, 'lkm543', self.miner_rewards)
    new_block.hash = self.get_hash(new_block, 0) #產生創世塊當前雜湊值
    self.chain.append(new_block) #將創世塊加入鏈中

* 放置交易明細至新區塊中

In [29]:
class BlockChain:
  #交易明細加入新區塊中
  def add_transaction_to_block(self, block):
    #Get the transaction with highest fee by block_limitation
  
    #將等待中的所有交易明細一手續費大小排序，並反序陣列使第一個元素為手續費最高之交易明細
    self.pending_transactions.sort(key=lambda x: x.fee, reverse=True)
  
    #檢查等待中交易明細數量是否超載區塊容量
    if len(self.pending_transactions) > self.block_limitation:
      transaction_accepted = self.pending_transactions[:self.block_limitation] 
      self.pending_transactions = self.pending_transactions[self.block_limitation:] #留下不被接受的交易明細在等待區
    else:
      transaction_accepted = self.pending_transactions #接受全部等待中的交易明細
      self.pending_transactions = [] #重至等待區

    #放入區塊中
    block.transactions = transaction_accepted

* 挖掘新區塊

In [30]:
class BlockChain:
  #利用"工作量證明"挖掘新區塊
  def mine_block(self, miner):
    start = time.process_time() #紀錄挖掘前時間

    #產生新區塊
    last_block = self.chain[-1] #選取鏈中最後一區塊
    new_block = Block(last_block.hash, self.difficulty, miner, self.miner_rewards) #設定新區塊參數
    self.add_transaction_to_block(new_block) #加入交易明細至新區塊
    new_block.previous_hash = last_block.hash
    new_block.difficulty = self.difficulty
    new_block.hash = self.get_hash(new_block, new_block.nonce) #產生加入交易明細後的新雜湊值

    #透過改變nonce值得到新雜湊值，如符合難度定義"開頭有幾個0"則為合格的雜湊值與nonce值
    while new_block.hash[0: self.difficulty] != '0' * self.difficulty:
      new_block.nonce += 1
      new_block.hash = self.get_hash(new_block, new_block.nonce)
    
    #計算並得出區塊挖掘時間花費
    time_consumed = round(time.process_time() - start, 5)
    print(f"Hash found: {new_block.hash} @ difficulty {self.difficulty}, time cost: {time_consumed}s") #顯示新區塊狀態提示

    #將所挖掘的新區塊加入鏈中
    self.chain.append(new_block)